In [1]:
import pandas as pd

# Load the weather data CSV
weather_data = pd.read_csv('Year_weather.csv')  # Replace with the actual file path

# Load the fuel mix data CSV
fuel_data = pd.read_csv('genfuelmix_aggregatedyear.csv')  # Replace with the actual file path

# Convert 'datetime' in weather data to datetime format
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# Convert 'datetime' in weather data to UTC to match the fuel data time zones
weather_data['datetime'] = weather_data['datetime'].dt.tz_localize('UTC')

# Print the 'BeginDate' column for inspection
print("BeginDate column before conversion:")
print(fuel_data['BeginDate'].head())

# Convert 'BeginDate' in fuel data to datetime format with explicit UTC conversion
fuel_data['BeginDate'] = pd.to_datetime(fuel_data['BeginDate'], errors='coerce', utc=True)

# Print the 'BeginDate' column after conversion to check if conversion was successful
print("BeginDate column after conversion:")
print(fuel_data['BeginDate'].head())

# Check for any invalid dates (if any)
invalid_dates = fuel_data[fuel_data['BeginDate'].isna()]
if not invalid_dates.empty:
    print("Invalid 'BeginDate' entries found:")
    print(invalid_dates)
    # Optionally, drop rows with invalid 'BeginDate'
    fuel_data = fuel_data.dropna(subset=['BeginDate'])

# Round 'BeginDate' in the fuel data to the nearest hour
fuel_data['rounded_hour'] = fuel_data['BeginDate'].dt.round('h')  # Use 'h' instead of 'H'

# Sort both DataFrames by time for 'merge_asof' to work
fuel_data = fuel_data.sort_values('rounded_hour')
weather_data = weather_data.sort_values('datetime')

# Perform the merge_asof to match the nearest hour in the weather data for each fuel data entry
combined_data = pd.merge_asof(fuel_data, weather_data, left_on='rounded_hour', right_on='datetime', direction='backward')

# Drop the 'rounded_hour' column since it's no longer needed
combined_data = combined_data.drop(columns=['rounded_hour'])
combined_data = combined_data.sort_values('BeginDate')

# Save the combined data to a new CSV file
combined_data.to_csv('combined_fuel_weather_data.csv', index=False)

print("Data combined and saved to 'combined_fuel_weather_data.csv'")


BeginDate column before conversion:
0    2022-10-01 00:00:13-04:00
1    2022-10-01 00:03:46-04:00
2    2022-10-01 00:07:39-04:00
3    2022-10-01 00:10:24-04:00
4    2022-10-01 00:14:19-04:00
Name: BeginDate, dtype: object
BeginDate column after conversion:
0   2022-10-01 04:00:13+00:00
1   2022-10-01 04:03:46+00:00
2   2022-10-01 04:07:39+00:00
3   2022-10-01 04:10:24+00:00
4   2022-10-01 04:14:19+00:00
Name: BeginDate, dtype: datetime64[ns, UTC]
Invalid 'BeginDate' entries found:
       BeginDate  Coal  Hydro  Natural Gas  Nuclear  Oil  Other  Landfill Gas  \
174847       NaT   0.0  216.0       7513.0   1226.0  0.0   63.0          20.0   
174848       NaT   0.0  214.0       7630.0   1228.0  0.0    0.0          20.0   
174849       NaT   0.0  214.0       7630.0   1228.0  0.0    0.0          20.0   
174850       NaT   0.0  211.0       7562.0   1228.0  0.0    2.0          20.0   
174851       NaT   0.0  212.0       7318.0   1228.0  0.0   17.0          20.0   
...          ...   ...    ..